### 1. Load Libraries

In [95]:
import numpy as np
import pandas as pd

import data

%matplotlib inline


### 2. Load Dataset

In [96]:
# Extract and retrieve rentals data from Microsoft SQL server
# Refer to documentation within data module for technical and configuration details
df_rentals = data.get_rentals()

df_rentals.head()

,date,hr,weather,temperature,feels_like_temperature,relative_humidity,windspeed,psi,guest_scooter,registered_scooter
0,2012-07-02,23,clear,109.0,140.0,51.0,7.0,13,37,631
1,2011-12-22,10,cloudy,80.2,109.4,82.0,6.0,35,41,894
2,2011-02-25,11,clear,90.4,120.2,77.0,30.0,30,27,350
3,2012-03-10,1,clear,71.8,95.0,36.0,17.0,40,2,354
4,2011-06-19,5,cloudy,102.2,132.8,78.0,0.0,1,23,82


### 3. Data Insights <a id='data_insights'></a>

In [6]:
df_rentals.shape

(18643, 10)

- Dataset contains 18,643 observations with 10 features.


- There are 24 hours a day, 365 days a year. So over 2 years, there should be a maximum 17,520 (24 x 365 x 2) observations.


- Given that there are more hourly observations than hours over a 2 year period, some of the observations may be duplicates or erroneous. 


- The problem statement is to predict the total number of active e-scooter users given the above dataset.


- Each observation records the number of guest and registered users using rental e-scooters in a particular hour of a day.


- I shall assume that the total number of active e-scooter users in a particular hour of a day is the sum of the guest and registered users <i><b>i.e. total active users = guest users + registered users.</b></i>

In [39]:
df_rentals.columns.values

array(['date', 'hr', 'weather', 'temperature', 'feels_like_temperature',
       'relative_humidity', 'windspeed', 'psi', 'guest_scooter',
       'registered_scooter'], dtype=object)

- Column labels of the rentals dataset

In [97]:
df_rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18643 entries, 0 to 18642
Data columns (total 10 columns):
date                      18643 non-null object
hr                        18643 non-null int64
weather                   18643 non-null object
temperature               18643 non-null float64
feels_like_temperature    18643 non-null float64
relative_humidity         18643 non-null float64
windspeed                 18643 non-null float64
psi                       18643 non-null int64
guest_scooter             18643 non-null int64
registered_scooter        18643 non-null int64
dtypes: float64(4), int64(4), object(2)
memory usage: 1.4+ MB


- No column with null/missing value.

### 4. Summary Statistics

In [55]:
df_rentals.describe()

,hr,temperature,feels_like_temperature,relative_humidity,windspeed,psi,guest_scooter,registered_scooter
count,18643.000000,18643.000000,18643.000000,18643.000000,18643.000000,18643.000000,18643.000000,18643.000000
mean,11.537145,88.433037,117.313608,62.733251,12.741082,25.142198,106.388940,1074.471383
std,6.924281,16.252200,20.364081,19.315897,8.217008,14.442978,147.151664,1055.916934
min,0.000000,48.100000,60.800000,0.000000,0.000000,0.000000,-2.000000,-2.000000
25%,6.000000,75.200000,100.400000,48.000000,7.000000,13.000000,12.000000,240.000000
50%,12.000000,88.700000,118.400000,63.000000,13.000000,25.000000,50.000000,807.000000
75%,18.000000,102.200000,134.600000,78.000000,17.000000,38.000000,144.000000,1535.500000
max,23.000000,131.000000,179.600000,100.000000,57.000000,50.000000,1099.000000,6203.000000


- Large differnece in the 75th %tile and max values of columns <b>windspeed</b>, <b>guest_scooter</b>, <b>registered_scooter</b>
- This observation suggests that there are extreme values or outliers in these columns. 

### 5. Data Cleaning

#### 5.1 <b style="font-family:'Courier New'; font-size:18px">date</b> Column

In [113]:
# Check data type of date column
df_rentals.dtypes['date']

dtype('O')

- Convert the <b style="font-family:'Courier New'; font-size:15px">date</b> column from <b style="font-family:'Courier New'; font-size:15px">string</b> to <b style="font-family:'Courier New'; font-size:15px">date</b> data type.


- Combine the <b style="font-family:'Courier New'; font-size:15px">date</b> and <b style="font-family:'Courier New'; font-size:15px">hr</b> columns to a <b style="font-family:'Courier New'; font-size:15px">datetime</b> column.
- This is to facilitate the use of datetime/timeseries operations when doing exploration and feature engineering later.

In [98]:
# Rename date column to date_str to indicate string data type
df_rentals.rename(columns={'date': 'date_str'}, inplace=True)

# Convert date column from string to datetime data type
# df_rentals['date'] = pd.to_datetime(df_rentals['date'])

# Verify column data type
# df_rentals.dtypes['date']

In [99]:
# Create datetime column by concatenating the date and hr columns
df_rentals['datetime'] = df_rentals.apply(lambda row: row.date_str + ' ' + str(row.hr), axis=1) + ':00'

# Convert datetime column from string to datetime data type
df_rentals.datetime = pd.to_datetime(df_rentals.datetime)

# Verify column data type
df_rentals.dtypes['datetime']

dtype('<M8[ns]')

#### 5.2 <b style="font-family:'Courier New'; font-size:18px">weather</b> Column

In [109]:
df_rentals.weather.unique()

array(['clear', 'cloudy', 'light snow/rain', 'heavy snow/rain'],
      dtype=object)

- The <b>weather</b> column contains categorical data.
- The <b>weather</b> data is 'dirty', clean up is neccessary. 
- Mixed cases i.e. clear and CLEAR..
- Incorrect spelling e.g. lear, clar


- Correct values 'lear' and 'clar' to be 'clear'.
- Correct values 'cludy' and 'loudy' to be 'cloudy'.
- Correct value 'liht snow/rain' to be 'light snow/rain'.

In [101]:
# Standardized weather column to lower case characters
df_rentals.weather = df_rentals.weather.str.lower()

In [108]:
dict_weather = {

    # Replace incorrect values 'lear' and 'clar' with 'clear'
    'lear': 'clear',
    'clar': 'clear',

    # Replace incorrect values 'cludy' and 'loudy' with 'cloudy'
    'cludy': 'cloudy',
    'loudy': 'cloudy',

    # Replace incorrect value 'liht snow/rain' with 'light snow/rain'
    'liht snow/rain': 'light snow/rain'
}

# Replace incorrect values in weather column
df_rentals.replace({'weather': dict_weather}, inplace=True)

In [110]:
# Verify that the incorrect values have been replaced
df_rentals.weather.unique()

array(['clear', 'cloudy', 'light snow/rain', 'heavy snow/rain'],
      dtype=object)

- The <b>weather</b> column contains 4 unique categorical values i.e. clear, cloudy, light snow/rain and heavy snow/rain.
- One-hot encoding can be applied to the <b>weather</b> column later in feature engineering.

#### 5.3 <b style="font-family:'Courier New'; font-size:18px">temperature</b>, <b style="font-family:'Courier New'; font-size:18px">feels_like_temperature</b> Columns

In [17]:
# Get the maximum and minimum temperature recorded
max(df_rentals.temperature), min(df_rentals.temperature)

(131.0, 48.1)

In [18]:
# Get maximum and minimum feels_like_temperature recorded
max(df_rentals.feels_like_temperature), min(df_rentals.feels_like_temperature)

(179.6, 60.8)

In [19]:
# Number of observations with temperatures above 120°F
len(df_rentals[df_rentals.temperature > 120])

240

- I shall assume that values from the <b>temperature</b> and <b>feels_like_temperature</b> columns are in fahrenheit.

- I shal assume that this dataset is gathered from a city/town since people are renting e-scooters and e-bikes.


- The maximum value of the <b>temperature</b> column is 131°F which is pretty close to the [highest temperature ever recorded](https://en.wikipedia.org/wiki/List_of_weather_records#Highest_temperatures_ever_recorded) of 134.1°F.


- According to [TripSavvy](https://www.tripsavvy.com/the-worlds-hottest-cities-4070053), some of the highest temperatures recorded in a city include Phoenix 122°F, Marrakech 120°F, Mecca 121.6°F, Kuwait City 126°F, Ahvaz 129°F and Timbuktu 120°F.


- There are 240 observations with temperatures above 120°F.  This dataset should be from a city known for its high temperatures. If otherwise, the temperatures in these observations need to be verified.


- 'Feels like' temperature is also known as the [heat index](https://en.wikipedia.org/wiki/Heat_index).  In short, it is a temperature reading that factors in a component of relative humidity.


- We can verify the values of the <b>feels_like_temperature</b> column using the heat index [formula](https://en.wikipedia.org/wiki/Heat_index#Formula).


- Without any geographical information on this dataset given, I shall assume that all temperature readings are accurate. 

#### 5.4 <b style="font-family:'Courier New'; font-size:18px">relative_humidity</b> Column

In [20]:
# Get the maximum and minimum values of relative humidity recorded
max(df_rentals.relative_humidity), min(df_rentals.relative_humidity)

(100.0, 0.0)

In [111]:
# Number of observations with 0 relative humidity
len(df_rentals[df_rentals.relative_humidity==0])

25

- [Relative humidity](https://en.wikipedia.org/wiki/Relative_humidity) (RH) is the actual amount of water vapor present in relation to the capacity that the air has at a particular temperature.  It is express as a percentage.


- A relative humidity reading of 0 implies [air devoid of water vapor](https://www.chicagotribune.com/news/ct-xpm-2011-12-16-ct-wea-1216-asktom-20111216-story.html). This is quite impossible given the climate conditions of a city/town, where I assume this dataset is gathered.  Values of 0 in the <b>relative_humidity</b> column need to be verified.


- Since there are only 25 observations with 0 relative humidity, I've decided to drop them.


- A relative humidity reading of 100 means that the air is totally saturated with water vapor and cannot hold any more, creating the possibility of rain.  So values of 100 in the <b>relative_humidity</b> column are valid.

In [112]:
# Number of observations in dataset
len(df_rentals)

18643

In [113]:
# Drop observations with relative humidity value of 0
df_rentals.drop(df_rentals[df_rentals.relative_humidity==0].index, inplace=True)

# Check number of observations left after dropping
len(df_rentals)

18618

#### 5.5 <b style="font-family:'Courier New'; font-size:18px">windspeed</b> Column


In [29]:
# Get the maximum and minimum values of the windspeed column
max(df_rentals.windspeed), min(df_rentals.windspeed)

(57.0, 0.0)

- No units were given for the <b>windspeed</b> column.


- Apparently, wind speed can be measured using a variety of [units](https://en.wikipedia.org/wiki/Wind_speed#Units) e.g. beaufort, knots, m/s, km/h, mph, depending on purpose, region or target audience.


- [Wind speed of 0](https://www.wral.com/weather/blogpost/1116592/) is possible and said to be calm.


- I'm unable to gauge if the maximum wind speed of 57 is valid. 57 m/s implies a hurricane, but 57 km/h is just a near gale.  


- As such, I shall assume that values in the <b>windspeed</b> column are valid.


#### 5.6 <b style="font-family:'Courier New'; font-size:18px">psi</b> Column

In [33]:
# Get the maximum and minimum values of the psi column
max(df_rentals.psi), min(df_rentals.psi)

(50, 0)

- The [Pollutant Standard Index (psi)](https://en.wikipedia.org/wiki/Pollutant_Standards_Index) is a measure of pollutants present in the air (0 to 400). 


- Values in the <b>psi</b> column are valid.

#### 5.7 <b style="font-family:'Courier New'; font-size:18px">guest_scooter</b>, <b style="font-family:'Courier New'; font-size:18px">registered_scooter</b> Columns

In [44]:
# Get the maximum and minimum values of the guest_scooter column
max(df_rentals.guest_scooter), min(df_rentals.guest_scooter)

(1099, -2)

In [45]:
# Get the maximum and minimum values of the registered_scooter column
max(df_rentals.registered_scooter), min(df_rentals.registered_scooter)

(6203, -2)

In [46]:
# Number of observations with a negative value in either the guest_scooter or registered_scooter columns
len(df_rentals[(df_rentals.guest_scooter<0) | (df_rentals.registered_scooter<0)])

658

- Values in the <b>guest_scooter</b> and <b>registered_scooter</b> columns indicate the number of guest and registered users renting e-scooters in a particular hour, of a particular date.


- As such, the values in the <b>guest_scooter</b> and <b>registered_scooter</b> columns should not be negative.


- There are 658 observations with a negative value in either the <b>guest_scooter</b> or <b>registered_scooter</b> columns.


- As there is no way of verifying these erroneous values, I shall set all negative values in the <b>guest_scooter</b> or <b>registered_scooter</b> columns to 0.

In [56]:
# Set all negative values in the guest_scooter column to 0
df_rentals.loc[df_rentals.guest_scooter < 0, 'guest_scooter'] = 0

# Set all negative values in the registered_scooter column to 0
df_rentals.loc[df_rentals.registered_scooter < 0, 'registered_scooter'] = 0

# Verify that there all negative values in the guest_scooter and registered_scooter columns have been set to 0
len(df_rentals[(df_rentals.guest_scooter<0) | (df_rentals.registered_scooter<0)])

0

#### 5.8 Duplicate Observations

In [76]:
# Number of observations in dataset
len(df_rentals)

18618

- As mentioned in section 3. [Data Insights](#data_insights), there are more hourly observations than hours over 2 years from 2011 to 2012.


- There are 18,618 hourly observations versus 17,520 (24 x 365 x 2) hours in the years 2011 and 2012.


- Therefore, there are duplicate or erroneous observations in the dataset.

In [74]:
# Number of observations that are duplicates
len(df_rentals[df_rentals.duplicated()])

1609

- There are 1,609 duplicate observations in the dataset.

 
- I shall drop these duplicated observations.

In [81]:
# Drop duplicate observations
df_rentals.drop_duplicates(inplace=True)

# Verify that the duplicate observations have been removed
len(df_rentals), any(df_rentals.duplicated())

(17009, False)

### 6. Feature Engineering

In [93]:
# Create active_scooter column as target variable
df_rentals['active_scooter'] = df_rentals.guest_scooter + df_rentals.registered_scooter

In [94]:
df_rentals

,date_str,hr,weather,temperature,feels_like_temperature,relative_humidity,windspeed,psi,guest_scooter,registered_scooter,datetime,active,active_scooter
7010,2011-01-01,0,clear,66.7,95.0,81.0,0.0,50,7,90,2011-01-01 00:00:00,97,97
18246,2011-01-01,1,clear,65.0,93.2,80.0,0.0,6,23,224,2011-01-01 01:00:00,247,247
9198,2011-01-01,2,clear,65.0,93.2,80.0,0.0,25,14,188,2011-01-01 02:00:00,202,202
1786,2011-01-01,3,clear,66.7,95.0,75.0,0.0,41,9,70,2011-01-01 03:00:00,79,79
7448,2011-01-01,4,clear,66.7,95.0,75.0,0.0,9,2,11,2011-01-01 04:00:00,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14909,2012-12-31,19,cloudy,68.4,91.4,60.0,11.0,15,34,760,2012-12-31 19:00:00,794,794
6675,2012-12-31,20,cloudy,68.4,91.4,60.0,11.0,25,26,572,2012-12-31 20:00:00,598,598
13354,2012-12-31,21,clear,68.4,91.4,60.0,11.0,35,21,585,2012-12-31 21:00:00,606,606
1191,2012-12-31,22,clear,68.4,93.2,56.0,9.0,8,41,337,2012-12-31 22:00:00,378,378
